# Audiobooks business case

### Extract the data from the csv

In [1]:
import numpy as np
from sklearn import preprocessing

# Load the data
raw_csv_data = np.loadtxt('Audiobooks_data.csv',delimiter=',')

unscaled_inputs_all = raw_csv_data[:,1:-1]
targets_all = raw_csv_data[:,-1]

### Shuffle the data

In [2]:
shuffled_indices = np.arange(unscaled_inputs_all.shape[0])
np.random.shuffle(shuffled_indices)

unscaled_inputs_all = unscaled_inputs_all[shuffled_indices]
targets_all = targets_all[shuffled_indices]

### Balance the dataset

In [3]:
num_one_targets = int(np.sum(targets_all))

zero_targets_counter = 0

indices_to_remove = []

for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)

unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis=0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis=0)

### Standardize the inputs

In [4]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

In [5]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

### Split the dataset into train, validation, and test

In [6]:
samples_count = shuffled_inputs.shape[0]

train_samples_count = int(0.8 * samples_count)
validation_samples_count = int(0.1 * samples_count)

test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

1828.0 3579 0.5107571947471361
207.0 447 0.46308724832214765
202.0 448 0.45089285714285715


### Save the three datasets in *.npz

In [7]:
# Save the three datasets in *.npz.

np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Audiobooks_data_test', inputs=test_inputs, targets=test_targets)

In [8]:
npz = np.load('Audiobooks_data_train.npz')

train_inputs = npz['inputs'].astype(np.float64)
train_targets = npz['targets'].astype(np.int32)

npz = np.load('Audiobooks_data_validation.npz')

validation_inputs = npz['inputs'].astype(np.float64)
validation_targets = npz['targets'].astype(np.int32)

npz = np.load('Audiobooks_data_test.npz')

test_inputs = npz['inputs'].astype(np.float64)
test_targets = npz['targets'].astype(np.int32)

In [9]:
# Model
import tensorflow as tf

output_size = 2
hidden_layer_size = 50

model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [10]:
batch_size = 50
max_epochs = 100
early_stopping = tf.keras.callbacks.EarlyStopping(patience=3)

model.fit(train_inputs, train_targets, epochs=max_epochs, callbacks=[early_stopping], validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/100
112/112 - 1s - loss: 0.5086 - accuracy: 0.7429 - val_loss: 0.4341 - val_accuracy: 0.7808 - 616ms/epoch - 5ms/step
Epoch 2/100
112/112 - 0s - loss: 0.4069 - accuracy: 0.7843 - val_loss: 0.4159 - val_accuracy: 0.7785 - 109ms/epoch - 970us/step
Epoch 3/100
112/112 - 0s - loss: 0.3860 - accuracy: 0.7941 - val_loss: 0.4386 - val_accuracy: 0.7875 - 110ms/epoch - 981us/step
Epoch 4/100
112/112 - 0s - loss: 0.3808 - accuracy: 0.7958 - val_loss: 0.3845 - val_accuracy: 0.7964 - 112ms/epoch - 1ms/step
Epoch 5/100
112/112 - 0s - loss: 0.3711 - accuracy: 0.8080 - val_loss: 0.3912 - val_accuracy: 0.7897 - 115ms/epoch - 1ms/step
Epoch 6/100
112/112 - 0s - loss: 0.3715 - accuracy: 0.7974 - val_loss: 0.3859 - val_accuracy: 0.7852 - 109ms/epoch - 976us/step
Epoch 7/100
112/112 - 0s - loss: 0.3656 - accuracy: 0.8055 - val_loss: 0.3785 - val_accuracy: 0.7897 - 111ms/epoch - 990us/step
Epoch 8/100
112/112 - 0s - loss: 0.3643 - accuracy: 0.8044 - val_loss: 0.3706 - val_accuracy: 0.8098 - 111ms/e

In [11]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

14/14 [==============================] - 0s 1ms/step - loss: 0.3394 - accuracy: 0.8259
Test loss: 0.34. Test accuracy: 82.59%
